In [ ]:
#Program scrapping FAKE and REAL news data from POLITIFACT.com and use as Dataset. 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [ ]:

authors = []
dates = []
statements = []
source = []
targets = []

In [4]:
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')

    statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer'})
    statement_quote = soup.find_all('div', attrs= {'class': 'm-statement__quote'})
    statement_meta = soup.find_all('div', attrs={'class': 'm-statement__meta'}) 
    target = soup.find_all('div', attrs={'class':'m-statement__meter'})